In [1]:
# import libraries
import pandas as pd
import numpy as np
import string


In [2]:
############################ PREPARATION #########################

In [2]:
# read stopwords, positive, negative, negation wordlist
s_col_names = ['stp']
s_words = pd.read_csv(
    'data/dea/stopwords.csv',
    header=None,
    names=s_col_names,
    low_memory=False)

p_words = pd.read_csv(
    'data/dea/positive-words.csv',
    low_memory=False)

n_words = pd.read_csv(
    'data/dea/negative-words.csv',
    low_memory=False)

nt_words = pd.read_csv(
    'data/dea/negation-words.csv',
    low_memory=False)

do_col_names = ['do']
do_words = pd.read_csv(
    'data/dea/product-do.csv',
    header=None,
    names=do_col_names,
    low_memory=False)

rm_names = ['rm']
remwords = pd.read_csv(
    'data/dea/removes-words.csv',
    header=None,
    names=rm_names,
    low_memory=False)

# make list from dataframes
stopwords = [str(y) for y in s_words.stp]
positive_words = [str(x) for x in p_words.pos_]
negative_words = [str(y) for y in n_words.neg_]
negation_words = [str(y) for y in nt_words.netion_]
proddo_words = [str(y) for y in do_words.do]
rem_words = [x for x in remwords.rm]

In [3]:
import string
string_punc = string.punctuation.replace(',', '')
string_punc = string_punc + "’"
string_punc

'!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~’'

In [4]:
# replace the apostrophe from the lists
def ReplaceApostrophe(thelist):
    newlist = []
    for word in thelist:
        word = word.replace("'", '')
        newlist.append(word)
    return newlist

stopwords = list(set(ReplaceApostrophe(stopwords)))
negative_words = list(set(ReplaceApostrophe(negative_words)))
negation_words = list(set(ReplaceApostrophe(negation_words)))
positive_words = list(set(ReplaceApostrophe(positive_words)))
proddo_words = list(set(ReplaceApostrophe(proddo_words)))


In [6]:
############################ THE SENTIMENT #########################

In [5]:
# The Sentiment Analysis Class
import re
# import enchant
# weng = enchant.Dict("en_US")

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer

porter_stemmer = PorterStemmer()

def CheckSentenceScore(sentence):
    # check words
    negatives = []
    positives = []
    negations = []
    proddo = []        
    # value gived by words type
    score_pos = 1
    score_neg = -1

    negation = neg = pos = False
    sum_score = 0

    for word in sentence:
        word = reduce_lengthening(word)
        if word in positive_words:
            if negation is True:
                sum_score += score_neg
                negation = False
                pos = True
            elif pos is True:
                pos = False
            else:
                sum_score += score_pos        
                pos = True
            positives.append(word)
            

        elif word in negative_words:
            if neg is True:
                sum_score += score_pos
            elif neg is True:
                neg = False
            elif pos is True:
                sum_score += (-2)
                pos = False
            elif negation is True:
                sum_score += score_pos
                negation = False
            else:
                sum_score += score_neg
            neg = True
            negatives.append(word) 

        elif word in negation_words:
            pos = neg = False
            negation = True
            negations.append(word)

        elif word in proddo_words:
            if negation is True:
                sum_score += score_neg
                negation = False
            proddo.append(word) 
            pos = neg =  False
        else:
            pos = neg = negation = False
            
    return positives, negatives, negations, proddo, sum_score


In [8]:
from nltk import tokenize
from nltk.tokenize import word_tokenize 
import re
from nltk.stem.porter import PorterStemmer

porter_stemmer = PorterStemmer()
corpus = list(set().union(positive_words, negative_words, negation_words, proddo_words))
# token by . ! and 
review = "Omg I am so enjoying this. Sitting around the back deck with family wishing we have music that doesn't require plugs or wires. Saw it online deals of the day read the reviews and ordered on he spot and drove a half hour drive to my closet Best Buy. Play it now everyday."
listp = tokenize.sent_tokenize(review)
scores = []
positive_list = []
negative_list = []
negation_list = []
proddo_list = []
for i in listp:
    print (i)
    print(listp.index(i))
    # remove all punctuation
    review = i.translate(str.maketrans('','',string_punc))

    # make entire text lowercase, remove numbers
    review = review.lower()
    review = re.sub(r'[0-9]+', '', review)
    word_tokens = word_tokenize(review)
    # remove some words
    remstop_sentence = [w for w in word_tokens if not w in stopwords]
    filtered_sentence = [w for w in remstop_sentence if not w in rem_words]
#     print(filtered_sentence)
    the_sentence = []
    for w in filtered_sentence:
        if w not in corpus:
            w = porter_stemmer.stem(w)
        the_sentence.append(w)
    print(the_sentence)
    positive_list, negative_list, negation_list, proddo_list, score = CheckSentenceScore(the_sentence)
    print("positives : "+str(positive_list))
    print("negatives : "+str(negative_list))
    print("negations : "+str(negation_list))
    print("product do : "+str(proddo_list))
    print("score: "+str(score))
    scores.insert(listp.index(i), score)
    
print(scores)
print(sum(scores))



Omg I am so enjoying this.
0
['omg', 'enjoy']
positives : ['enjoy']
negatives : []
negations : []
product do : []
score: 1
Sitting around the back deck with family wishing we have music that doesn't require plugs or wires.
1
['sit', 'around', 'back', 'deck', 'famili', 'wish', 'music', 'doesnt', 'requir', 'plug', 'wire']
positives : []
negatives : []
negations : ['doesnt']
product do : []
score: 0
Saw it online deals of the day read the reviews and ordered on he spot and drove a half hour drive to my closet Best Buy.
2
['saw', 'onlin', 'deal', 'day', 'review', 'and', 'order', 'spot', 'and', 'drove', 'half', 'hour', 'drive', 'closet']
positives : []
negatives : []
negations : []
product do : []
score: 0
Play it now everyday.
3
['play', 'everyday']
positives : []
negatives : []
negations : []
product do : ['play']
score: 0
[1, 0, 0, 0]
1


In [257]:
# the class trying
# The Sentiment Analysis Class
import re
# import enchant
# weng = enchant.Dict("en_US")

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer

porter_stemmer = PorterStemmer()


# value gived by words type
score_pos = 1
score_neg = -1

negation  = neg = passing = False
sum_score = 0

# check words
negatives = []
positives = []
negations = []
proddo = []     

def SentimentAnalysis(testimony):
    testimony_value = []
       

    for review in testimony:
        
        word_tokens = word_tokenize(review)
        # remove some words, stemming
        remstop_sentence = [w for w in word_tokens if not w in stopwords]

        # stemming
        stemmed_sentence = [porter_stemmer.stem(w) for w in remstop_sentence]

        # remove unuse words
        filtered_sentence = [w for w in stemmed_sentence if not w in rem_words]

        # value gived by words type
        score_pos = 1
        score_neg = -1

        negation  = pos = neg = passing = False
        sum_score = 0

        for word in filtered_sentence:
            word = reduce_lengthening(word)
            if word in positive_words:
                if negation is True:
                    sum_score += score_neg
                    negation = False
                elif pos is True:
                    pos = False
                else:
                    sum_score += score_pos

                positives.append(word)
                pos = True

            elif word in negative_words:
                if neg is True:
                    sum_score += score_pos
                elif pos is True:
                    sum_score += (-2)
                    pos = False
                elif negation is True:
                    sum_score += score_pos
                    negation = False
                else:
                    sum_score += score_neg
                neg = True
                negatives.append(word) 

            elif word in negation_words:
                pos = neg = False
                negation = True
                negations.append(word)

            elif word in proddo_words:
                if negation is True:
                    sum_score += score_neg
                    negation = False
                proddo.append(word) 
                pos = neg = False
            else:
                pos = neg = negation = False
        testimony_value.append(sum_score)
    return testimony_value

In [11]:
"and" in stopwords

False

In [259]:
# get the data from csv file to dataframe
DataTest = pd.read_csv(
    'data/dea/try100data.csv', 
    low_memory=False)

In [260]:
DataTest.head()

,reviewId,testimony,hum_sentiment_score
0,941,A lazy mans drean when it is combined with Ale...,1
1,942,I really enjoy my Fire stick. It's really easy...,2
2,943,Really cool device! Instantly noticed the diff...,1
3,944,Love it! Works great. One in each of the main ...,2
4,765,This is a great addition to the other Alexa pr...,7


In [261]:
# save testimony in a list
TestTestimony = [str(x) for x in DataTest.testimony]
# remove all punctuation
TestTestimony = [testimony.translate(str.maketrans('','',string_punc)) for testimony in TestTestimony ]
# make entire text lowercase
TestTestimony = [t.lower() for t in TestTestimony]
# remove numbers
TestTestimony = [re.sub(r'[0-9]+', '', t) for t in TestTestimony]


In [262]:
sentiment_score = SentimentAnalysis(TestTestimony)
DataTest['sentiment_score'] = sentiment_score
DataTest.head()

,reviewId,testimony,hum_sentiment_score,sentiment_score
0,941,A lazy mans drean when it is combined with Ale...,1,1
1,942,I really enjoy my Fire stick. It's really easy...,2,2
2,943,Really cool device! Instantly noticed the diff...,1,1
3,944,Love it! Works great. One in each of the main ...,2,2
4,765,This is a great addition to the other Alexa pr...,7,7


In [263]:
DataTest['check_mechine'] = np.where(DataTest['sentiment_score'] != DataTest['hum_sentiment_score'], False , True)
(DataTest['check_mechine']==False).sum()

4

In [264]:
DataTest[DataTest['check_mechine']==False]

,reviewId,testimony,hum_sentiment_score,sentiment_score,check_mechine
11,743,The Show is a great little product. It has ope...,2,1,False
67,3791,I believe that this will be very useful during...,1,0,False
92,2585,We bought the fire for my 85 yr old aunt. She ...,2,1,False
96,5041,i had tv for 10 months and lost pic and sound ...,1,2,False


In [99]:
import pandas as pd
import numpy as np

DataTest10 = pd.read_csv(
    'data/training-testing/DataTest10wSentScore.csv', 
    low_memory=False)

In [100]:
DataTest10.columns

Index(['Unnamed: 0', 'reviewId', 'Unnamed: 0.1', 'itemId', 'userId', 'rating',
       'sentiment_score_norm', 'sentiment_score', 'hum_sentiment_score',
       'testimony_update'],
      dtype='object')

In [103]:
DataTest10.head()

,reviewId,itemId,userId,rating,sentiment_score,hum_sentiment_score,testimony_update
0,2314,7,75,5,3,3,This is great for learning for little kids. Ea...
1,5032,27,433,5,0,0,Is full myself for watching TV
2,4735,1,3300,5,1,1,This is the third Fire tablet I have bought. T...
3,3097,11,3479,3,5,2,When it works it's great but for some reason i...
4,1469,20,291,5,4,4,I love being able to turn my lights on or off ...


In [104]:
test_sentiment_score_norm = ScoreNormalization(DataTest10.sentiment_score)
DataTest10['sentiment_score_norm'] = test_sentiment_score_norm
DataTest10.head()

,reviewId,itemId,userId,rating,sentiment_score,hum_sentiment_score,testimony_update,sentiment_score_norm
0,2314,7,75,5,3,3,This is great for learning for little kids. Ea...,4.5
1,5032,27,433,5,0,0,Is full myself for watching TV,3.0
2,4735,1,3300,5,1,1,This is the third Fire tablet I have bought. T...,3.5
3,3097,11,3479,3,5,2,When it works it's great but for some reason i...,5.0
4,1469,20,291,5,4,4,I love being able to turn my lights on or off ...,5.0


In [102]:
DataTest10.drop(["Unnamed: 0", "Unnamed: 0.1", "sentiment_score_norm"], 1, inplace = True)

In [105]:
test_sentiment_score_norm = ScoreNormalization(DataTest10.hum_sentiment_score)
DataTest10['hum_sentiment_score_norm'] = test_sentiment_score_norm
DataTest10.head()

,reviewId,itemId,userId,rating,sentiment_score,hum_sentiment_score,testimony_update,sentiment_score_norm,hum_sentiment_score_norm
0,2314,7,75,5,3,3,This is great for learning for little kids. Ea...,4.5,4.5
1,5032,27,433,5,0,0,Is full myself for watching TV,3.0,3.0
2,4735,1,3300,5,1,1,This is the third Fire tablet I have bought. T...,3.5,3.5
3,3097,11,3479,3,5,2,When it works it's great but for some reason i...,5.0,4.0
4,1469,20,291,5,4,4,I love being able to turn my lights on or off ...,5.0,5.0


In [106]:
len(DataTest10)

112

In [107]:
n = len(DataTest10)
for index, row in DataTest10.iterrows():
#     print(row[4], row[8])
    sum_ = 0
    sum_ = sum_ + float(abs(row[4]-row[8]))
    
MAE = float(sum_) / n
print(MAE)

0.017857142857142856


In [108]:
# DataTest10.to_csv("data/training-testing/DataTest10wSentScore.csv")

In [1]:
import re

string = 'asdfkweni12311sdkvne'
result = re.sub(r'[0-9]+', '', string)
result

'asdfkwenisdkvne'